# 4. Analysis of two sources in the same field of view - 1D analysis

## 4.1. Context
This notebook displays the most advanced capabilities of the one-dimensional analysis with DL3 data and `Gammapy` and is also a preview of the capability we would like to achieve with the three-dimensional analysis: to analyse several sources in the field of view. We will use observations of the blazar [1ES1218+304](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=1ES+1218%2B304&utm_source=chatgpt.com) gathered in 2018 and 2019. Using the same dataset, we also analyze the blazar [1ES1215+303](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=1ES+1215%2B303&utm_source=chatgpt.com). These two sources are separated by only 0.8 degrees, which is smaller than the MAGIC field of view radius (1.5 degrees).

In [ ]:
# - basic imports (numpy, astropy, regions, matplotlib)
from pathlib import Path
import astropy.units as u
from astropy.coordinates import SkyCoord
from regions import PointSkyRegion, CircleSkyRegion
import matplotlib.pyplot as plt
import logging
import warnings

# - Gammapy's imports
from gammapy.maps import MapAxis, WcsGeom, RegionGeom
from gammapy.data import DataStore
from gammapy.datasets import SpectrumDataset, Datasets
from gammapy.makers import (
    SpectrumDatasetMaker,
    WobbleRegionsFinder,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
    EBLAbsorptionNormSpectralModel,
)
from gammapy.modeling import Fit
from gammapy.estimators import (
    FluxPointsEstimator,
)
from plot_utils import plot_on_off_regions_skymap, plot_gammapy_sed

# - setting up logging and ignoring warnings
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

warnings.filterwarnings("ignore")

Let us load all the data first.

In [ ]:
# load observations
datastore = DataStore.from_dir("../acme_magic_odas_data/data/1ES1218+304/point_like/")
observations = datastore.get_observations(required_irf=["rad_max", "aeff", "edisp"])

## 4.2. 1ES1218+304 analysis

The observations in this data set were performed wobbling around the coordinates of 1ES1218+304, with the standard offset (for MAGIC observations) of $0.4^{\circ}$. Let us first compute the spectrum and the light curve for this source. As we saw in the previous notebooks, the _off_ regions are placed symmetrically to the _on_ region with respect to the camera center, so they will sit on a circle of radius $0.4^{\circ}$ around it. There is something to be taken into account in this particular case, though. The source 1ES1215+303, an active gamma-ray emitter, sits in this very same field of view. We want to be sure that none of the _off_ regions coincides with 1ES1215+303, otherwise our background estimation would be contaminated by its gamma rays. All these processes are automatically accounted for by `Gammapy` in the data reduction.

### 4.2.1. Sources, signal, background, and exclusion regions visualization

Let us try to visualise the problem first. Let us plot a sky map for each of the wobble pointings, and mark on it the position of the source of intereset (1ES1218+304), the position of the other source in the field of view (1ES1215+303), and the _on_ and _off_ regions.

In [ ]:
# coordinates of the two sources
_1ES1215_coordinates = SkyCoord.from_name("1ES1215+303", frame="icrs")
_1ES1218_coordinates = SkyCoord.from_name("1ES1218+304", frame="icrs")

# center of the ON region
on_region_circle_1ES1218 = CircleSkyRegion(
    center=_1ES1218_coordinates,
    radius=0.2 * u.deg,
)

# create an exclusion mask over 1ES1215+303, in order not to have overlapping OFF regions
exclusion_region_1ES1215 = CircleSkyRegion(
    center=_1ES1215_coordinates,
    radius=0.2 * u.deg,
)

# create the geometry for the skymaps
# create also the energy axes needed for the geometry
skymap_geom_1ES1218 = WcsGeom.create(
    skydir=_1ES1218_coordinates, binsz=0.05, width="4 deg", frame="icrs", proj="TAN"
)

I created a function, `plot_on_off_regions_skymap` in `plot_utils.py`, to illustrate how the positions of the two sources is changing in the camera frame and how we are placing the _on_, _off_, and exclusion regions. 

In [ ]:
for observation in (observations[0], observations[2], observations[5], observations[7]):
    plot_on_off_regions_skymap(
        observation=observation,
        skymap_geom=skymap_geom_1ES1218,
        on_region=on_region_circle_1ES1218,
        exclusion_region=exclusion_region_1ES1215,
        n_off_regions=3,
        source_name="1ES1218+304",
        excluded_source_name="1ES1215+303",
    )

All the maps are centered on the source of interest (1ES1218+304). By looking at these four observations, we can cleary see how the telescope is "wobbling" around it. In the second-to-last observation plotted, we can clearly see the problem we were describing: one of the _off_ regions overalps with the second source (1ES1215+303). This will be accounted for in the data reduction, by creating an _exclusion mask_, let us do it.

### 4.2.2. Spectrum extraction and fit

Note that we will be using a different _on_ `Region` and a different `RegionGeom` w.r.t. those that we used to create the sky map.
In particular, remember that if the $\theta^2$ is defined in the `RAD_MAX` table, then the _on_ region must be a `PointSkyRegion` (its size will be taken from the `RAD_MAX` table).
Also, we have to give an energy axis to the `RegionGeom`, as we will be extracting a spectrum.

In [ ]:
# before the Dataset creation, define the energy axes
energy_min = 10 * u.GeV
energy_max = 1e5 * u.GeV
n_energy_est_bins = 20
n_energy_true_bins = 28

energy_axis = MapAxis.from_energy_bounds(
    energy_min,
    energy_max,
    n_energy_est_bins,
    per_decade=False,
    unit="GeV",
    name="energy",
)
energy_true_axis = MapAxis.from_energy_bounds(
    energy_min,
    energy_max,
    n_energy_true_bins,
    per_decade=False,
    unit="GeV",
    name="energy_true",
)

Compared to the previous data reduction, we have to specify an additional parameter when we compute the background: an **exclusion region** corresponding to the source that is not of our interest (1ES1215+303). We can create it as a circular region of radius 0.2 deg around its coordinates (which we already defined to plot the black star in the sky maps). We have to convert it to a pixel mask. We do so by _blacking out_ the pixels of the skymap geometry that fall within the exclusion region.

In [ ]:
exclusion_mask_1ES1215 = ~skymap_geom_1ES1218.region_mask([exclusion_region_1ES1215])

In [ ]:
# use an on region that is now a PointSkyRegion
on_region_point_1ES1218 = PointSkyRegion(_1ES1218_coordinates)
# define the geometry for the spectrum extraction
geom = RegionGeom.create(region=on_region_point_1ES1218, axes=[energy_axis])

# create an empty dataset
dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_true_axis)

# dataset maker
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, selection=["counts", "exposure", "edisp"]
)
# NOTE: use 2 off regions to avoid overlapping with the exclusion region
region_finder = WobbleRegionsFinder(n_off_regions=2)
# NOTE: remember to specify the exclusion region in the background maker
bkg_maker = ReflectedRegionsBackgroundMaker(
    region_finder=region_finder, exclusion_mask=exclusion_mask_1ES1215
)

datasets_1ES1218 = Datasets()

for observation in observations:
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(observation.obs_id)), observation
    )
    dataset_on_off = bkg_maker.run(dataset, observation)
    datasets_1ES1218.append(dataset_on_off)

In [ ]:
# let us stack the dataset for the spectrum computation
dataset_1ES1218_stacked = datasets_1ES1218.stack_reduce()

# set the proper energy range for fitting proper minimum and maximum energy for the fit, display the result
e_min = 60 * u.GeV
e_max = 10 * u.TeV
dataset_1ES1218_stacked.mask_fit = dataset_1ES1218_stacked.counts.geom.energy_mask(
    e_min, e_max
)

# let us ckeck what we produced
dataset_1ES1218_stacked.peek()

Let us now try to fit the spectrum with a power law model.

In [ ]:
# assign the model
spectral_model_1ES1218 = PowerLawSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    index=2,
    reference=200 * u.GeV,
)
# enlarge range of amplitude parameter
# to facilitate UL computation
spectral_model_1ES1218.parameters["amplitude"].min = 1e-15
spectral_model_1ES1218.parameters["amplitude"].max = 1e-7

# define a sky model
model_1ES1218 = SkyModel(spectral_model=spectral_model_1ES1218, name="1ES1218+304")
dataset_1ES1218_stacked.models = [model_1ES1218]

fit = Fit()
results = fit.run(datasets=dataset_1ES1218_stacked)
print(results)
print(spectral_model_1ES1218)

In [ ]:
# compute flux points
energy_edges = energy_axis.edges[4:14]
flux_points_estimator_1ES1218 = FluxPointsEstimator(
    energy_edges=energy_edges, source="1ES1218+304", selection_optional="all"
)
flux_points_1ES1218 = flux_points_estimator_1ES1218.run(
    datasets=dataset_1ES1218_stacked
)

In [ ]:
# make a plot of the spectrum
fig, ax = plt.subplots()
plot_gammapy_sed(
    ax,
    spectral_model_1ES1218,
    flux_points_1ES1218,
    color="crimson",
    label="1ES1218+304",
)
ax.legend()
plt.show()

Quite a steep spectrum: we have obtained an index of 3 (see `PowerLawSpectralModel` above). We also observe that the flux points don't fall exactly on top of the broad band model, as if there was more curvature to them.   
We remember that on their way to Earth, gamma rays are absorbed by the Extragalactic Background Light (EBL), and this effect is more pronounced at higher energies. We can try to fit the spectrum with a power law model absorbed by the EBL, using one of the models implemented in `Gammapy`. Let us do it.

In [ ]:
# Load the EBL model. Here we use the model from Dominguez, 2011
redshift_1ES1218 = 0.182
absorption_1ES1218 = EBLAbsorptionNormSpectralModel.read_builtin(
    "dominguez", redshift=redshift_1ES1218
)

fig, ax = plt.subplots()
absorption_1ES1218.plot(ax=ax, energy_bounds=[e_min, e_max])
ax.set_ylim([1e-4, 2])
plt.show()

Quite a lot of absorption going on, already a factor 10 at 1 TeV!

In [ ]:
# compute spectrum and spectral points
absorbed_spectral_model_1ES1218 = spectral_model_1ES1218 * absorption_1ES1218
# define a new sky model and assign it to the data
absorbed_model_1ES1218 = SkyModel(
    spectral_model=absorbed_spectral_model_1ES1218, name="1ES1218+304"
)
dataset_1ES1218_stacked.models = [absorbed_model_1ES1218]

# re-perform the forward folding
results = fit.run(datasets=dataset_1ES1218_stacked)
print(results)
print(absorbed_model_1ES1218)

# re-compute also flux points
absorbed_flux_points_1ES1218 = flux_points_estimator_1ES1218.run(
    datasets=dataset_1ES1218_stacked
)
# get the intrinsic ones, i.e. those without the absorption factor
deabsorbed_flux_points_1ES1218 = absorbed_flux_points_1ES1218.copy(
    reference_model=SkyModel(spectral_model=spectral_model_1ES1218)
)

By fitting the spectrum with a model that accounts for the EBL absorption, we can now obtain an _intrinsic_ or _deabsorbed_ spectrum, i.e. an estimation of the actual flux emitted from the source.
Let us plot both.

In [ ]:
# make a plot of the spectrum
fig, ax = plt.subplots()
plot_gammapy_sed(
    ax,
    spectral_model_1ES1218,
    deabsorbed_flux_points_1ES1218,
    "crimson",
    "1ES1218+304, intrinsic",
)
plot_gammapy_sed(
    ax,
    absorbed_spectral_model_1ES1218,
    absorbed_flux_points_1ES1218,
    "dodgerblue",
    "1ES1218+304, absorbed",
)
ax.legend()
ax.set_ylim([1e-13, 1e-10])
plt.show()

## 4.3. 1ES1215+303 analysis

Would it be possible now to analyse the other blazar in the field (1ES1215+303) with the same dataset?   
As we said before, the observations in this data set were conducted wobbling around the coordinates of 1ES1218+304,   
This will imply that 1ES1215+303 will be observed at different offsets in the camera, depending on the wobble position.


### 4.3.1. 1ES1215+303 projected coordinates and multi-offset IRFs
Let us use the function `plot_source_camera_offsets` in `plot_utils.py` to visualize the position of 1ES1215+303 in the camera frame for each of the four wobble pointings.   
Let us now also switch the roles of the two sources, and draw a circular exclusion region of radius 0.2 deg around 1ES1218+304, to avoid contamination from this source when estimating the background for 1ES1215+303.

In [ ]:
# center of the ON region
on_region_circle_1ES1215 = CircleSkyRegion(
    center=_1ES1215_coordinates,
    radius=0.2 * u.deg,
)

# create an exclusion mask over 1ES1218+304, in order not to have overlapping OFF regions
exclusion_region_1ES1218 = CircleSkyRegion(
    center=_1ES1218_coordinates,
    radius=0.2 * u.deg,
)

# create the geometry and add the energy axes
# geometry to be used in all the skymaps
skymap_geom_1ES1215 = WcsGeom.create(
    skydir=_1ES1215_coordinates, binsz=0.05, width="5 deg", frame="icrs", proj="TAN"
)
exclusion_mask_1ES1218 = ~skymap_geom_1ES1215.region_mask([exclusion_region_1ES1218])

n_off_regions = 3

In [ ]:
for observation in (observations[0], observations[2], observations[5], observations[7]):
    plot_on_off_regions_skymap(
        observation=observation,
        skymap_geom=skymap_geom_1ES1215,
        on_region=on_region_circle_1ES1215,
        exclusion_region=exclusion_region_1ES1218,
        n_off_regions=3,
        source_name="1ES1215+303",
        excluded_source_name="1ES1218+304",
    )

What is clear, is that, per each of the wobble pointings, 1ES1215+303 sits at a different offset from the camera center.
Let compute these offsets.

In [ ]:
offsets = []
for obs in (observations[0], observations[2], observations[5], observations[7]):
    offset = obs.pointing.get_icrs().separation(_1ES1215_coordinates)
    offsets.append(offset)

offsets.sort()

print(f"offsets = {offsets}")

As in the first notebook, the IRFs that we have been using up to now are computed for a single offset: the source's separation from the camera center (0.4 deg, as that is the standard offset of most MAGIC observations). 

These _single-offset_ IRF is no longer useful for this new case, as we need to know the response of our system at different offsets. For such cases, MAGIC produces _multi-offset_ IRFs, that is IRFs computed at several offsets from the camera center. `Gammapy` is able to handle these multi-offset IRFs and to interpolate them at the required offset. Let us check, using the effective area, that the offsets we computed for 1ES1215+303 are indeed included in the multi-offset IRFs that we have in our DL3 data.

In [ ]:
observations[0].aeff.plot();

In [ ]:
observations[0].aeff.plot_energy_dependence(offset=offsets)
plt.yscale("log")

### 4.3.2. 1ES1215+303 Spectrum extraction and fit
At the data reduction level, the only difference w.r.t. the previous analysis is that the definition of the exclusion mask now has to be centered on 1ES1218+304. The rest of the analysis is exactly the same as before. `Gammapy` will take care of interpolating the IRFs at the required offsets for each of the runs.

In [ ]:
# use an on region that is now a PointSkyRegion
on_region_point_1ES1215 = PointSkyRegion(_1ES1215_coordinates)
geom = RegionGeom.create(region=on_region_point_1ES1215, axes=[energy_axis])
dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_true_axis)

dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, selection=["counts", "exposure", "edisp"]
)
# use 2 off regions to avoid overlapping with the exclusion region
region_finder = WobbleRegionsFinder(n_off_regions=2)
# NOTE: remember to specify the background maker in the exclusion region
bkg_maker = ReflectedRegionsBackgroundMaker(
    region_finder=region_finder, exclusion_mask=exclusion_mask_1ES1218
)

datasets_1ES1215 = Datasets()

for observation in observations:
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(observation.obs_id)), observation
    )
    dataset_on_off = bkg_maker.run(dataset, observation)
    datasets_1ES1215.append(dataset_on_off)

In [ ]:
# let us stack the dataset for the spectrum computation
dataset_1ES1215_stacked = datasets_1ES1215.stack_reduce()

# set the proper energy range for fitting proper minimum and maximum energy for the fit, display the result
e_min = 60 * u.GeV
e_max = 10 * u.TeV
dataset_1ES1215_stacked.mask_fit = dataset_1ES1215_stacked.counts.geom.energy_mask(
    e_min, e_max
)

dataset_1ES1215_stacked.peek()

Let us directly take into account the EBL absorption in this case.

In [ ]:
# assign the model
spectral_model_1ES1215 = PowerLawSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    index=2,
    reference=200 * u.GeV,
)
# enlarge range of amplitude parameter
# to facilitate UL computation
spectral_model_1ES1215.parameters["amplitude"].min = 1e-15
spectral_model_1ES1215.parameters["amplitude"].max = 1e-7

# Load the EBL model. Here we use the model from Dominguez, 2011
redshift_1ES1215 = 0.131
absorption_1ES1215 = EBLAbsorptionNormSpectralModel.read_builtin(
    "dominguez", redshift=redshift_1ES1215
)

absorbed_spectral_model_1ES1215 = spectral_model_1ES1215 * absorption_1ES1215

# define a sky model
model_1ES1215 = SkyModel(
    spectral_model=absorbed_spectral_model_1ES1215, name="1ES1215+303"
)
dataset_1ES1215_stacked.models = [model_1ES1215]

fit = Fit()
results = fit.run(datasets=dataset_1ES1215_stacked)
print(results)
absorbed_spectral_model_1ES1215.parameters.to_table()

In [ ]:
# compute flux points
energy_edges = energy_axis.edges[5:13]
flux_points_estimator_1ES1215 = FluxPointsEstimator(
    energy_edges=energy_edges, source="1ES1215+303", selection_optional="all"
)
absorbed_flux_points_1ES1215 = flux_points_estimator_1ES1215.run(
    datasets=dataset_1ES1215_stacked
)
# get the intrinsic ones, i.e. those without the absorption factor
deabsorbed_flux_points_1ES1215 = absorbed_flux_points_1ES1215.copy(
    reference_model=SkyModel(spectral_model=spectral_model_1ES1215)
)

Let us now plot both sources with their respective absorbed and intrinsic spectra.

In [ ]:
fig, ax = plt.subplots()

plot_gammapy_sed(
    ax,
    absorbed_spectral_model_1ES1218,
    absorbed_flux_points_1ES1218,
    "crimson",
    "1ES1218+304, absorbed",
)
plot_gammapy_sed(
    ax,
    spectral_model_1ES1218,
    deabsorbed_flux_points_1ES1218,
    "rosybrown",
    "1ES1218+304, intrinsic",
)

plot_gammapy_sed(
    ax,
    absorbed_spectral_model_1ES1215,
    absorbed_flux_points_1ES1215,
    "dodgerblue",
    "1ES1215+303, absorbed",
)
plot_gammapy_sed(
    ax,
    spectral_model_1ES1215,
    deabsorbed_flux_points_1ES1215,
    "lightskyblue",
    "1ES1215+303, intrinsic",
)

ax.set_xlim([60, 2e4])
ax.legend()
plt.show()

Let us save the best-fit models and the flux points to disk for use in the next tutorial.

In [ ]:
results_dir_1ES1218 = Path("results/1ES1218+304")
results_dir_1ES1218.mkdir(exist_ok=True, parents=True)

results_dir_1ES1215 = Path("results/1ES1215+303")
results_dir_1ES1215.mkdir(exist_ok=True, parents=True)

dataset_1ES1218_stacked.models.write(
    results_dir_1ES1218 / "model_1ES1218+304.yaml", overwrite=True
)
dataset_1ES1215_stacked.models.write(
    results_dir_1ES1215 / "model_1ES1215+303.yaml", overwrite=True
)

absorbed_flux_points_1ES1215.write(
    results_dir_1ES1215 / "flux_points_1ES1215+303.fits",
    sed_type="e2dnde",
    overwrite=True,
)
absorbed_flux_points_1ES1218.write(
    results_dir_1ES1218 / "flux_points_1ES1218+304.fits",
    sed_type="e2dnde",
    overwrite=True,
)

### Exercise 4.1.
Fit the spectrum of 1ES1218+303 with a log parabola model absorbed by the EBL. Is the fit better than with a simple power law?

### Exercise 4.2. (homework)
Compute the light curve of both sources. Which one is more variable?